# Amazon_data

## Part 1: Database and Jupyter Notebook Set Up
### Using following command to import dataset into mongo_db:

 mongoimport --type csv -d Clean_Data_Resources -c amazon_cleanData --headerline --drop amazon_cleanData.csv

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
import pandas as pd
from collections import Counter
import re

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
print(mongo.list_database_names())

In [ ]:
# assign the Clean_Data_Resources database to a variable name
db = mongo['Clean_Data_Resources']

In [ ]:
# review the collections in our amazon database
print(db.list_collection_names())

In [ ]:
# review a document in the establishments collection
#Display the first document in the results using pprint before filter
pprint(db.amazon_cleanData.find_one())

In [ ]:
# assign the collection to a variable
amazon = db['amazon_cleanData']

In [ ]:
# Display the total number of items in the amazon collection
amazon.count_documents({})

### Analysis

Based on Jupyter Notebook analysis, we know that there are 8 unique values (unique category column names) in the 'category' column which is the data collection of different kinds of cable in our amazon dataset. I am going to use 8 different unique values to make some queries , in order to derive some interesting information/facts and understand the trends of different cables being sold in the amazon website.

In [ ]:
#Creating some queries to understand the trends of cables available in the dataset
#  Create a query that finds all computer USB cables in the 'amazon' collection
# Filter results by name
query = {'category': 'ComputersAccessoriesAccessoriesPeripheralsCablesAccessoriesCablesUSBCables'}

# Capture the results to a variable
results1 = amazon.find(query)

# Print the number of results
Computer_USBs = int(amazon.count_documents(query))
print("Total Number of items which are computer USB cables in result are:", Computer_USBs)


In [ ]:
## Pretty print the results
pprint(results1[0:10])

#displaying results1
for i in results1:
    print(i)

In [ ]:
#   Create a query that finds top 5 computer USB cables in the 'amazon' collection which has discounted price greater than equal to 100 rupees and have rating score greater than equal to 4
# Filter results by name
#query = {'category': 'ComputersAccessoriesAccessoriesPeripheralsCablesAccessoriesCablesUSBCables',
        #'discounted_price': {'$gte': 100}, 'rating':{'$gte':4}}
query = {'category': 'ComputersAccessoriesAccessoriesPeripheralsCablesAccessoriesCablesUSBCables',
        'rating_count': {'$gte': 10000}, 'rating':{'$gte':4}}


# Display the  fields
fields = {'category' :1, 'product_id':1, 'product_name': 1 , 'discounted_price': 1, 'discount_percentage':1,'actual_price': 1, "rating":1,"review_title":1, "review_content":1, "rating_count":1}

# sort in descending order by 
#sort = [('discounted_price', -1),('discount_percentage', -1),]
sort = [('rating_count', - 1)]

# limit the results to the first 5
limit = 5

#  Execute the query
result2= (list(amazon.find(query, fields).sort(sort).limit(limit)))


In [ ]:
#displaying the result
result2 

In [ ]:
# Initialize an empty list to store the top 5 computer USB Cables which has discounted price greater than equal to 100 and have ratings greater than equal to 4
comp_USB_OVER_100 = []

# Loop through each document in the results and append it to the list
for item in result2:
   comp_USB_OVER_100.append(item)
  
# Now, converted_results is a list of dictionaries
pprint(comp_USB_OVER_100)

In [ ]:
# Create a DataFrame from the list 
df_Top_comp_usb_over_100 = pd.DataFrame(comp_USB_OVER_100)

# Display the DataFrame
print("The number of rows of the dataframe is:", len(df_Top_comp_usb_over_100))
print(df_Top_comp_usb_over_100)

In [ ]:
#displaying dataframe containing top 5 computer USB Cables which has discounted price greater than equal to 100 and have ratings greater than equal to 4
df_Top_comp_usb_over_100

In [ ]:
#checking column names
print(df_Top_comp_usb_over_100.columns)

In [ ]:
# Reorder the columns
Reordered_Top_comp_usb_over_100 = df_Top_comp_usb_over_100[[
    "product_id", "product_name", "category", "actual_price", "discounted_price",
    "discount_percentage", "rating", "review_title", "review_content","rating_count"
]]

# Print the DataFrame
print("The Top  5 computer USB Cables which has discounted price greater than equal to 100 and have ratings greater than equal to 4 are following:")
Reordered_Top_comp_usb_over_100

In [ ]:
# Creating a chart that shows top 5 computer USB Cables which has discounted price greater than equal to 100 and have ratings greater than equal to 4.
filtered_df = Reordered_Top_comp_usb_over_100

fig = px.bar(filtered_df, x = 'product_id', y = 'rating_count',
             hover_data= ['product_name', 'actual_price', 'discount_percentage'], color = 'discount_percentage',
             labels = {'rating_count': 'Rating_Counts'}, height = 400) 
# Update layout for a clearer x-axis and set the title
fig.update_layout(
    title="Top 5 Computer USBs with discounted price >= 100 INR and Rating Score >=4"
)  
                             

fig.show()

In [ ]:
#saving the chart into the output Viz folder 
fig.write_html('../outputs_Viz/Most_popular_computer_USBs.html')



## Analysis

Based on the above chart and dataframe consisting of top 5 Computer USBs with discounted price >= 100 INR and Rating Score >=4 , we can see that, **product id "B086JTMRYL"** **ESR USB C to Lightning Cable,** is the most popular computer USB Cables having 19,763 rating counts, even though it is offering the lowest discounted percentage among other USB Cables competitors . Whereas, both of the  **Belkin  Cables** (product ids  "B084MZXJNK" and "B084N133Y7") are the least popular ones , both having only 1,951 rating counts . Also, although the **Amazon Cables** has the highest actual price (2100 INR) among 5, it offers 52 % discount rate on the cables making the discounted price the lowest (999 INR) , probably to increase the market share or popularity of the product. However, the relatively lower rating counts (5492) suggests the product is either still new in the market or doesnot offer the same level of quality and satisfaction to its customers like ESR USB cables does. 
   On the other hand, the discounted price of ERC cables is second highest among all. And bearing in mind all these cables got '4' as rating score,  with the positive review content and title , it is not hard to guess why ERC cables are the most popular Computer USBs among all even though it doesnot offer the least price to its customers. 



 


In [ ]:
# Define the list of words to search for
words_to_find = ['loved', 'best', 'awesome', 'nice', 'amazing']

# Create a regex pattern to match any of the words
regex_pattern = '|'.join([f'(?i){word}' for word in words_to_find])  # '(?i)' for case-insensitivity

# Aggregation pipeline
pipeline = [
    {'$match': {'review_content': {'$regex': regex_pattern}}},
    {'$unwind': '$review_content'},
    {'$match': {'review_content': {'$regex': regex_pattern}}},
    {'$group': {'_id': '$product_id', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 5}
]

# Run the aggregation pipeline
top_products = list(amazon.aggregate(pipeline))

# Output the result
if top_products:
    print("Top 5 products with the most mentions of the specified words:")
    for product in top_products:
        print(f"Product ID: {product['_id']} - Mentions: {product['count']}")
else:
    print("No products reviews contained any of the specified words.")

    print("No product reviews contained any of the specified words.")


In [ ]:
import plotly.graph_objs as go

# Given output data
products_data = [
    {"Product ID": "B08R69VDHT", "Mentions": 3},
    {"Product ID": "B09C6HXFC1", "Mentions": 3},
    {"Product ID": "B07XLCFSSN", "Mentions": 3},
    {"Product ID": "B08WRWPM22", "Mentions": 3},
    {"Product ID": "B08CF3B7N1", "Mentions": 3}
]

# Extracting product IDs and mention counts
product_ids = [product['Product ID'] for product in products_data]
mention_counts = [product['Mentions'] for product in products_data]

# Creating the bar chart
fig = go.Figure(data=[
    go.Bar(x=product_ids, y=mention_counts, text=mention_counts, textposition='auto')
])

# Updating the layout of the chart
fig.update_layout(
    title='Top 5 Products With Most Mentions of Specific Words',
    xaxis=dict(title='Product ID'),
    yaxis=dict(title='Number of Mentions'),
    margin=dict(l=40, r=40, t=40, b=40),
    paper_bgcolor="LightSteelBlue",
)

# Display the figure
fig.show()


In [ ]:
import plotly.graph_objs as go

# Sample data: Top 5 products with mentions count (assuming equal distribution of words for demonstration)
products_data = [
    {"Product ID": "B08R69VDHT", "Mentions": 3, "Words": "loved: 1, best: 1, awesome: 1"},
    {"Product ID": "B09C6HXFC1", "Mentions": 3, "Words": "loved: 1, nice: 1, amazing: 1"},
    {"Product ID": "B07XLCFSSN", "Mentions": 3, "Words": "best: 2, nice: 1"},
    {"Product ID": "B08WRWPM22", "Mentions": 3, "Words": "awesome: 2, amazing: 1"},
    {"Product ID": "B08CF3B7N1", "Mentions": 3, "Words": "loved: 1, best: 1, nice: 1"}
]

# Extract product IDs and mention counts
product_ids = [product['Product ID'] for product in products_data]
mention_counts = [product['Mentions'] for product in products_data]
hover_texts = [product['Words'] for product in products_data]

# Create the bar chart
fig = go.Figure(data=[
    go.Bar(x=product_ids, y=mention_counts, text=mention_counts, textposition='auto', hoverinfo='text', hovertext=hover_texts)
])

# Update chart layout for better presentation
fig.update_layout(
    title='Top 5 Products With Most Mentions of Specific Words in Reviews',
    xaxis_title="Product ID",
    yaxis_title="Number of Mentions",
    hovermode='closest'
)

# Add interactive functionality: Display review words on hover
fig.update_traces(marker_color='lightblue', marker_line_color='blue',
                  marker_line_width=1.5, opacity=0.6)

# Display the figure
fig.show()


In [ ]:
from textblob import TextBlob
import textblob



In [ ]:
# Fetch documents
documents = amazon.find({}, {'review_content': 1, '_id': 0})  # Adjust field name as necessary
for document in documents:
    review_content = document.get('review_content', '')
    if review_content:  # Ensure there's text to analyze
        # Perform sentiment analysis
        analysis = TextBlob(review_content)
        # Print the sentiment
        print(f"Review: {review_content}\nSentiment: {analysis.sentiment}\n")


In [ ]:
Reordered_Top_comp_usb_over_100

In [ ]:
import textblob
#textblob.download_corpora()
from textblob import TextBlob

In [ ]:
# Sample DataFrame creation (replace this with your actual DataFrame)
#df = pd.DataFrame({'column_name': ['This is a great product', 'Worst purchase ever', 'I am satisfied']})

# Define a function to apply sentiment analysis on a text
def analyze_sentiment(text):
    testimonial = TextBlob(text)
    return testimonial.sentiment

# Apply the function to your column
Reordered_Top_comp_usb_over_100['sentiment'] = Reordered_Top_comp_usb_over_100['review_content'].apply(analyze_sentiment)

# The 'sentiment' column will now have polarity and subjectivity scores
print(Reordered_Top_comp_usb_over_100)

In [ ]:
# Function to return polarity and subjectivity as separate columns
def analyze_sentiment_polarity(text):
    return TextBlob(text).sentiment.polarity

def analyze_sentiment_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Apply the functions
Reordered_Top_comp_usb_over_100['polarity'] = Reordered_Top_comp_usb_over_100['review_content'].apply(analyze_sentiment_polarity)
Reordered_Top_comp_usb_over_100['subjectivity'] = Reordered_Top_comp_usb_over_100['review_content'].apply(analyze_sentiment_subjectivity)


In [ ]:
Reordered_Top_comp_usb_over_100

## Analysis

- sentiment: A tuple representing the sentiment analysis result, with the first element being polarity and the second being subjectivity. This column seems to be a summary of the following two columns and may not be necessary for further analysis.
- polarity: A numerical score within the range of -1 to 1, indicating the overall sentiment of the review content, where -1 is very negative, 0 is neutral, and 1 is very positive. This reflects the emotional leaning of the review text.
- subjectivity: A numerical score within the range of 0 to 1, indicating the level of subjectivity in the review content. A score close to 0 suggests the text is more objective (fact-based), while a score closer to 1 suggests it is more subjective (opinion-based).

### Analysis of the Sentiment Scores:

- General Sentiment: All listed products have positive sentiment scores (polarity > 0), indicating generally positive reviews. None of the scores are extremely high, suggesting that while the sentiment is positive, there may be room for improvement or that the reviews contain a mix of positive and negative sentiments.

- Subjectivity: The subjectivity scores are moderate, ranging from around 0.43 to 0.54. This suggests that the reviews contain a mix of personal opinions and objective facts. A higher subjectivity score indicates that customers are sharing personal experiences or opinions rather than stating objective facts about the products.

- Observations:
The products are all from categories related to cables and accessories, with similar sentiment scores, suggesting a general satisfaction among customers but with some variations in individual experiences.
The discount percentages and price reductions are significant, which might contribute to the positive sentiment by increasing the perceived value of the products.
The similarity in polarity and subjectivity scores for some products (for example, products 1 and 2 have identical scores, as do products 3 and 4) might indicate that customers have consistent experiences with these products or that the sentiment analysis captured common themes in the reviews.
Overall, the sentiment analysis output suggests that these products are well-received, with positive reviews outweighing negative ones. The moderate subjectivity scores imply that while personal opinions do influence the reviews, there is also a significant amount of factual content being discussed.






In [ ]:
#  Create a query that finds all TV HDMI cables in the 'amazon' collection
# Filter results by name
query = {'category': 'ElectronicsHomeTheater,TVVideoAccessoriesCablesHDMICables'}

# Capture the results to a variable
results3 = amazon.find(query)

# Print the number of results
TV_HDMI_cables = int(amazon.count_documents(query))
print("Total Number of items which are TV HDMI cables in result are:", TV_HDMI_cables)


In [ ]:
## Pretty print the 10 records
pprint(results3[0:10])

#displaying results1
for i in results3:
    print(i)

In [ ]:
#   Create a query that finds top 10  TV HDMI cables in the 'amazon' collection having discounted price greater than equal to 100 rupees and have rating score greater than equal to 4
# Filter results by name
query = {'category': 'ElectronicsHomeTheater,TVVideoAccessoriesCablesHDMICables',
        'discounted_price': {'$gte': 100}, 'rating':{'$gte':4}}

# Display the 'category', 'actual_price' and 'discounted_price' fields
fields = {'category' :1, 'product_id':1, 'product_name': 1 , 'discounted_price': 1, 'discount_percentage':1,'actual_price': 1, "rating":1,"review_title":1, "review_content":1, "rating_count":1}

# sort in descending order by 
sort = [('discounted_price', -1),('actual_price', -1), ('rating', -1)]

# limit the results to the first 10
limit = 10

# Pretty print the results
pprint(list(amazon.find(query, fields).sort(sort).limit(limit)))

In [ ]:
#displaying the result
result4 = list(amazon.find(query, fields).sort(sort))
result4

In [ ]:
# Initialize an empty list to store the top 10 TV HDMI Cables which has discounted price greater than equal to 100 and have ratings greater than equal to 4
top_tv_HDMI_OVER_100 = []

# Loop through each document in the results and append it to the list
for item in result4:
   top_tv_HDMI_OVER_100.append(item)
  
# Display the list
pprint(top_tv_HDMI_OVER_100)

In [ ]:
# Create a DataFrame from the list 
df_Top_TV_hdmi_over_100 = pd.DataFrame(top_tv_HDMI_OVER_100)

# Display the DataFrame
print(df_Top_TV_hdmi_over_100)

In [ ]:
df_Top_TV_hdmi_over_100.head(10)

In [ ]:
# Reorder the columns
Reordered_Top_tv_hdmi_over_100 = df_Top_TV_hdmi_over_100[[
    "product_id", "product_name", "category", "actual_price", "discounted_price",
    "discount_percentage", "rating", "review_title", "review_content","rating_count"
]]

# Print the first 10 rows of the DataFrame
print("The Top  10 TV HDMI Cables which has discounted price greater than equal to 100 and have ratings greater than equal to 4 are following:")
Reordered_Top_tv_hdmi_over_100.head(10)

In [ ]:
# Print the first 10 rows of the DataFrame
print("The Top  10 TV HDMI Cables which has discounted price greater than equal to 100 and have ratings greater than equal to 4 are following:")
Reordered_Top_tv_hdmi_over_100.head(10)

In [ ]:
#  Create a query that finds all TV USB cables in the 'amazon' collection
# Filter results by name

query = {'category': {'$regex': "cables$", '$options': 'i'}}

# Capture the results to a variable
result5 = amazon.find(query)

# Print the number of results
Total_cables = int(amazon.count_documents(query))
print("Total Number of cables in result are:",Total_cables  )


In [ ]:
## Pretty print the 10 records
Cables = []
pprint(result5)

#displaying results1
for i in result5:
    Cables.append(i)
   

In [ ]:
Cables[0:3]

In [ ]:
# Create a DataFrame from the list 
df_cables = pd.DataFrame(Cables)

#reorder the df_Cables dataframe
reordered_df_cables = df_cables[["category", "product_name", "discounted_price", "actual_price", "rating", "rating_count"]]

# Display the DataFrame
print("Total number of cables in the dataset are:" , int(len(df_cables)))
reordered_df_cables

In [ ]:
#  Create a query that finds all TV USB cables in the 'amazon' collection
# Filter results by name
query = {'category':  'ElectronicsHomeTheater,TVVideoAccessoriesCablesRCACables'}

# Capture the results to a variable
result6 = amazon.find(query)

# Print the number of results
RCA_cables = int(amazon.count_documents(query))
print("Total Number of rca cables in result are:", RCA_cables)


In [ ]:
#  Create a query that finds all optical cables in the 'amazon' collection
# Filter results by name
query = {'category':  'ElectronicsHomeTheater,TVVideoAccessoriesCablesOpticalCables'}

# Capture the results to a variable
result7 = amazon.find(query)

# Print the number of results
optical_cables = int(amazon.count_documents(query))
print("Total Number of optical cables in result are:", optical_cables)

In [ ]:
#  Create a query that finds all TV USB cables in the 'amazon' collection
# Filter results by name
query = {'category': 'ComputersAccessoriesAccessoriesPeripheralsCablesAccessoriesCablesDVICables'}

# Capture the results to a variable
result8 = amazon.find(query)

# Print the number of results
DVI_cables = int(amazon.count_documents(query))
print("Total Number of DVI cables in result are:", DVI_cables)

In [ ]:
#  Create a query that finds all TV USB cables in the 'amazon' collection
# Filter results by name
query = {'category':  'ElectronicsHomeTheater,TVVideoAccessoriesCablesSpeakerCables'}

# Capture the results to a variable
result9 = amazon.find(query)

# Print the number of results
Speaker_cables = int(amazon.count_documents(query))
print("Total Number of speaker cables in result are:", Speaker_cables)

In [ ]:
#  Create a query that finds all TV USB cables in the 'amazon' collection
# Filter results by name
query = {'category':   'ComputersAccessoriesAccessoriesPeripheralsCablesAccessoriesCablesEthernetCables'}

# Capture the results to a variable
result10 = amazon.find(query)

# Print the number of results
ethernet_cables = int(amazon.count_documents(query))
print("Total Number of ethernet cables in result are:", ethernet_cables)

In [ ]:
#  Create a query that finds all TV USB cables in the 'amazon' collection
# Filter results by name
query = {'category':   'ComputersAccessoriesAccessoriesPeripheralsCablesAccessoriesCablesSATACables'}

# Capture the results to a variable
results11 = amazon.find(query)

# Print the number of results
SATA_cables = int(amazon.count_documents(query))
print("Total Number of SATA cables in result are:",SATA_cables)

## Analysis

In [ ]:
print("There are three kinds of Cables in this dataset, they are:")
print("1. Computer USB Cables are:", Computer_USBs)
print("2. TV HDMI Cables are:" , TV_HDMI_cables)
print("3. RCA Cables are:" , RCA_cables)
print("4. Optical Cables are:" , optical_cables)
print("5. DVI Cables are:" , DVI_cables)
print("6. Speaker Cables are:" , Speaker_cables)
print("7. ethernet Cables are:" , ethernet_cables)
print("8. SATA Cables are:" , SATA_cables)
print("Therefore, Total Cables = Computer_USBs + TV_HDMI_cables + RCA_cables + optical cables + DVI cables + speaker cables + ethernet cables + SATA cables =", Total_cables)


In [ ]:
# Define the cable data
cable_data = {
    "Cable Type": [
        "Computer USB Cables",
        "TV HDMI Cables",
        "RCA Cables",
        "Optical Cables",
        "DVI Cables",
        "Speaker Cables",
        "Ethernet Cables",
        "SATA Cables"
    ],
    "Count": [231, 24, 2, 3, 1, 1, 2, 1]
}

# Create a DataFrame
df_cables_types = pd.DataFrame(cable_data)

# Calculate the total number of cables
df_cables_types["Total Cables"] = df_cables["Count"].sum()

# Display the DataFrame
df_cables_types

In [ ]:
# Creating a pie Chart from the cable dataframe
import plotly.graph_objects as go

labels = ["Computer USB Cables",
        "TV HDMI Cables",
        "RCA Cables",
        "Optical Cables",
        "DVI Cables",
        "Speaker Cables",
        "Ethernet Cables",
        "SATA Cables"]

values = [231, 24, 2, 3, 1, 1, 2, 1]

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[ 0.2, 0,0,0,0 ,0,0,0])])
fig.show()

In [ ]:
#saving the chart into the output folder 
fig.write_html('../outputs_Viz/Cables_pie.html')

In [ ]:
#  Create a query that finds all Smart Televisions in the 'amazon' collection
# Filter results by category name
query = {'category': 'ElectronicsHomeTheater,TVVideoTelevisionsSmartTelevisions'}

# Capture the results to a variable
result12 = amazon.find(query)

# Print the number of results
print("Total Number of items which are all Smart Televisions in result are:", amazon.count_documents(query))


In [ ]:
## Pretty print the 10 records
pprint(result12[0:10])

#displaying results1
for i in result12:
    print(i)


In [ ]:
#  Create a query that finds all Smart Televisions in the 'amazon' collection
# Filter results by category name
query = {'category': {'$regex': "SmartTelevisions"}}

# Capture the results to a variable
result13 = amazon.find(query)

# Print the number of results
print("Total Number of items which are all Smart Televisions in result are:", amazon.count_documents(query))


In [ ]:
#   Create a query that finds top 10  Smart Televisions in the 'amazon' collection have rating score greater than equal to 4 and highest rating count in descending order
# Filter results by product name
query = {'category': 'ElectronicsHomeTheater,TVVideoTelevisionsSmartTelevisions',
        'rating':{'$gte':4}}

# Display the 'category', 'actual_price' and 'discounted_price' fields
fields = {'category' :1, 'product_id':1, 'product_name': 1 , 'discounted_price': 1, 'discount_percentage':1,'actual_price': 1, "rating":1,"review_title":1, "review_content":1, "rating_count":1}

# sort in descending order by 
sort = [('rating', -1), ('rating_count', -1)]

limit = 10

# Using find() with sort and limit as chained method calls
pprint(list(amazon.find(query, fields).sort(sort).limit(limit)))

# Print the number of results
print("Total Number of items which are all Smart Televisions in result are:", amazon.count_documents(query))

In [ ]:
#displaying the result
result14 = list(amazon.find(query, fields).sort(sort).limit(limit))
result14


In [ ]:
# Initialize an empty list to store the top 10 TV HDMI Cables which has discounted price greater than equal to 100 and have ratings greater than equal to 4
top10_SmartTVs = []

# Loop through each document in the results and append it to the list
for item in result8:
   top10_SmartTVs.append(item)
  
# Display the list
pprint(top10_SmartTVs)


In [ ]:
# Create a DataFrame from the list 
df_Top_Smart_TV = pd.DataFrame(top10_SmartTVs)

# Display the DataFrame
#print(df_Top_Smart_TV)
df_Top_Smart_TV.head(10)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical Data Analysis

In [ ]:
# Calculating the correlation coefficient between 'discounted_price' and  "actual_price"
corr = df_Top_Smart_TV['actual_price'].corr(df_Top_Smart_TV['discounted_price'])
corr

# Analysis

The calculated correlation coefficient between 'actual_price' and 'discounted_price' is approximately 0.94. This value indicates a strong positive linear relationship between the actual price and the discounted price of items in the dataset. In practical terms, it means that as the actual price increases, the discounted price tends to increase as well, and vice versa. This strong correlation suggests that the two prices move in tandem to a large extent, implying that the discounting strategy might be proportionally related to the actual price of the items.






In [ ]:
# Calculating the correlation coefficient between  "rating_count" and 'discounted_price'
corr = df_Top_Smart_TV['rating_count'].corr(df_Top_Smart_TV['discounted_price'])
corr

In [ ]:
# Calculating the correlation coefficient between 'discounted_price' and  "rating_count"
corr = df_Top_Smart_TV['discounted_price'].corr(df_Top_Smart_TV['rating_count'])
corr

# Analysis


The calculated correlation coefficient between 'rating_count' and 'discounted_price' is approximately 0.054. This value indicates a very weak positive linear relationship between the number of ratings a product has received and its discounted price within the dataset. Essentially, this suggests that there is barely any linear correlation between how many times a product has been rated and its discounted price. The correlation is very close to 0, implying that changes in the discounted price of items do not significantly correspond with changes in their rating count, and vice versa.






In [ ]:
# Calculating the correlation coefficient between 'discounted_price' and  "actual_price"
corr = df_Top_Smart_TV['rating_count'].corr(df_Top_Smart_TV['rating'])
corr

In [ ]:
# Filter the DataFrame for TVs with a rating of 4
df_filtered = df_Top_Smart_TV[df_Top_Smart_TV['rating'] == 4]

# Transform the DataFrame to long format
df_long = df_filtered.melt(id_vars=['rating', 'product_id','product_name', 'category'], value_vars=['actual_price', 'discounted_price'],
                           var_name='Price Type', value_name='Price')


In [ ]:
df_long.head()

In [ ]:
# Create the bar chart using the long format DataFrame
fig = px.bar(df_long, x='Price Type', y='Price', color='Price Type', barmode='group', hover_data=['category','product_id','product_name' ])

# Update layout for a clearer x-axis and set the title
fig.update_layout(
    title="Discounted Price Vs Actual Prices for Smart TVs with Rating Score 4",
    xaxis_title="Price Type",
    yaxis_title="Price (in INR)",
    xaxis_tickangle=-45,
    legend_title="Price Type",
    # Make sure the currency format is correct for your locale
    # This example assumes that you are formatting the prices in Indian Rupees
)

# Show the figure
fig.show()

In [ ]:
#saving the chart into the output folder 
fig.write_html('../outputs_Viz/Amazon_bar1.html')


In [ ]:
# Create a pipeline that: 
# 1. Matches smart TVs having rating score greater than equal to 4
match_query = {'$match': {'category': 'ElectronicsHomeTheater,TVVideoTelevisionsSmartTelevisions',
                'rating': {'$gte': 4 }}}


# 2. Groups the matches by rating count having most rating count at the top
group_query = {'$group': {'_id':"$rating_count", 'total': {'$sum':1}}}

# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': {'total':-1}}

# Put the pipeline together
pipeline = [match_query, group_query, sort_values]

In [ ]:
# Run the pipeline through the aggregate method, cast the results as a list, and save the results to a variable
result15 = list(amazon.aggregate(pipeline))

In [ ]:
# Print the first 10 results
print("Number of  Smart TVS having rating score greater than equal to 4 in result are : ", len(result9))
print("The first ten results are: ")
pprint(result15[0:10])

# Analysis

### Here's a detailed explanation of the output list:

- {'_id': 32840, 'total': 5}: There are 5 smart TVs that received 32,840 ratings. This group has the highest number of TVs, indicating that 32,840 is a common rating count among the TVs considered.
- {'_id': 7109, 'total': 4}, {'_id': 4703, 'total': 4}: Each of these groups has 4 smart TVs that received 7,109 and 4,703 ratings, respectively.
- {'_id': 45238, 'total': 3}, {'_id': 16299, 'total': 3}, {'_id': 34899, 'total': 3}: Each of these groups consists of 3 smart TVs, with each group having 45,238, 16,299, and 34,899 ratings, respectively where the first one having the highest rating counts among all.
- Other entries ('total': 2): These represent groups where exactly 2 smart TVs have the specified number of ratings (_id values like 21252, 3587, etc.).
- This output essentially ranks the commonality of certain rating_count values among smart TVs in the dataset, starting from the most common (or shared by the most products) to less common. It's a valuable insight into the distribution of customer engagement (as measured by the count of ratings) across different products in the amazon data collection.

In [ ]:
# Convert the result to a Pandas DataFrame
smartTVS_df = pd.DataFrame(result9)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(smartTVS_df))

# Display the first 10 rows of the DataFrame
smartTVS_df.head(10)

In [ ]:
# Rename the columns
renamed_smartTVS_df = smartTVS_df.rename(columns={"_id": "rating_count",
                                              "total": "Total_count",
                                            })

# Print the first 10 rows of the DataFrame
print(" ")
renamed_smartTVS_df.head(10)

## Analysis


The summarized output of the DataFrame indicates a grouping of smart TVs based on their rating counts, revealing insights into customer engagement or popularity. Specifically, the top group consists of 5 smart TVs sharing a high rating count of 32,840, suggesting a significant level of customer interaction. Following this, there are groups of 4 smart TVs with rating counts of 7,109 and 4,703, respectively, indicating these are also highly rated items. Further down, groups of 3 smart TVs share rating counts of 45,238, 16,299, and 34,899, demonstrating  the highest level for the first one then moderate level of popularity for the second one but again higher level of popularity for the third one. Lastly, several groups of 2 smart TVs show, first one having relatively lower, yet notable, rating counts ranging from 1,611 to 21,252. This analysis underscores the variability in customer ratings among smart TVs, from extremely popular models with thousands of ratings to less popular ones with fewer ratings.

In [ ]:
#  Create a query that finds all TV HDMI cables in the 'amazon' collection
# Filter results by name
query = {'product_id': 'B09W5XR9RT'}

# Capture the results to a variable
result16 = amazon.find(query)

# Print the number of results
print("Total Number of items which are TV HDMI cables in result are:", amazon.count_documents(query11))